# Creating the inductance and capacitance matrix using GetDP and Gmsh  

In [17]:
import re

def read_pro_file(filename):
    with open(filename, 'r') as file:
        return file.read()

def write_pro_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)

def modify_constraints(content, active_electrode):
    pattern = r'(\s*\{ Region Electrode(\d+); Value (0|1)\.; \})'
    
    def replace_value(match):
        electrode_num = int(match.group(2))
        if electrode_num == active_electrode:
            return f"      {{ Region Electrode{electrode_num}; Value 1.; }}"
        else:
            return f"      {{ Region Electrode{electrode_num}; Value 0.; }}"
    
    return re.sub(pattern, replace_value, content)

def generate_pro_files(input_filename, num_electrodes):
    original_content = read_pro_file(input_filename)
    
    for i in range(1, num_electrodes + 1):
        new_content = modify_constraints(original_content, i)
        output_filename = f"electrode_{i}.pro"
        write_pro_file(output_filename, new_content)
        print(f"Generated {output_filename}")

# Usage
input_file = "wires_updated.pro"  # Replace with your input .pro file name
num_electrodes = 10
generate_pro_files(input_file, num_electrodes)

Generated electrode_1.pro
Generated electrode_2.pro
Generated electrode_3.pro
Generated electrode_4.pro
Generated electrode_5.pro
Generated electrode_6.pro
Generated electrode_7.pro
Generated electrode_8.pro
Generated electrode_9.pro
Generated electrode_10.pro


In [18]:
import subprocess
import os
import numpy as np

def run_getdp_and_process(pro_file, msh_file, getdp_path):
    # Run GetDP
    command = f"{getdp_path} {pro_file} -solve EleSta_v -pos Map -msh {msh_file}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error running GetDP for {pro_file}:")
        print(result.stderr)
        return None

    # Read the output file
    with open("output.txt", "r") as f:
        lines = f.readlines()

    # Extract charges
    charges = []
    for line in lines:
        parts = line.split()
        if len(parts) == 2:
            try:
                charges.append(float(parts[1]))
            except ValueError:
                print(f"Invalid charge value in line: {line}")
                return None

    # Delete the output file
    os.remove("output.txt")

    if len(charges) != 10:
        print(f"Expected 10 charge values, but found {len(charges)}")
        return None

    return charges

def generate_capacitance_matrix(pro_files, msh_file, getdp_path):
    num_electrodes = len(pro_files)
    capacitance_matrix = np.zeros((num_electrodes, num_electrodes))

    for i, pro_file in enumerate(pro_files):
        charges = run_getdp_and_process(pro_file, msh_file, getdp_path)
        if charges is None:
            return None
        capacitance_matrix[:, i] = charges

    return capacitance_matrix

def save_capacitance_matrix(matrix, filename):
    np.savetxt(filename, matrix, delimiter=',', fmt='%.6e')
    print(f"Capacitance matrix saved to {filename}")

def main():
    # File names
    pro_files = [f"electrode_{i}.pro" for i in range(1, 11)]
    msh_file = "wires.msh"  # Replace with your .msh file name
    output_file = "capacitance_matrix.txt"

    # GetDP executable path
    getdp_path = "/Users/renjuthomas/Downloads/getdp-3.5.0-MacOSX/bin/getdp"

    # Check if the provided path exists
    if not os.path.exists(getdp_path):
        print(f"Error: The specified path '{getdp_path}' does not exist.")
        print("Please make sure the GetDP executable is located at this path.")
        exit(1)

    # Generate capacitance matrix
    capacitance_matrix = generate_capacitance_matrix(pro_files, msh_file, getdp_path)

    if capacitance_matrix is not None:
        print("Capacitance Matrix:")
        print(capacitance_matrix)
        save_capacitance_matrix(capacitance_matrix, output_file)
    else:
        print("Failed to generate capacitance matrix. Please check the GetDP output for more details.")

if __name__ == "__main__":
    main()

Capacitance Matrix:
[[ 3.01239077e-11 -2.36058853e-11 -3.03354594e-12 -1.46838095e-12
  -8.24193757e-13 -4.84298143e-13 -2.90230526e-13 -1.77656365e-13
  -1.15047868e-13 -1.24668800e-13]
 [-2.36058853e-11  4.88289225e-11 -2.13141804e-11 -1.93186420e-12
  -8.54196564e-13 -4.65212472e-13 -2.71563975e-13 -1.64688713e-13
  -1.06294191e-13 -1.15036648e-13]
 [-3.03354594e-12 -2.13141804e-11  4.91621999e-11 -2.11387598e-11
  -1.83991329e-12 -8.00670801e-13 -4.35205207e-13 -2.57492773e-13
  -1.64738001e-13 -1.77693768e-13]
 [-1.46838095e-12 -1.93186420e-12 -2.11387598e-11  4.92206664e-11
  -2.10808815e-11 -1.81489172e-12 -7.89287623e-13 -4.34970970e-13
  -2.71494425e-13 -2.90135164e-13]
 [-8.24193757e-13 -8.54196564e-13 -1.83991329e-12 -2.10808815e-11
   4.92412643e-11 -2.10797082e-11 -1.81339630e-12 -8.00000211e-13
  -4.64959267e-13 -4.84015142e-13]
 [-4.84298143e-13 -4.65212472e-13 -8.00670801e-13 -1.81489172e-12
  -2.10797082e-11  4.92497258e-11 -2.10860397e-11 -1.83969483e-12
  -8.54635225

In [19]:
import numpy as np
from scipy import constants

def calculate_inductance_matrix(capacitance_matrix_file):
    # Load the capacitance matrix
    C = np.loadtxt(capacitance_matrix_file, delimiter=',')
    
    # Constants
    epsilon_0 = constants.epsilon_0  # Permittivity of free space
    mu_0 = constants.mu_0  # Permeability of free space
    c = constants.c  # Speed of light in vacuum
    
    # Calculate the inductance matrix
    L = mu_0 * epsilon_0 * np.linalg.inv(C)
    
    return L

def save_inductance_matrix(L, filename):
    np.savetxt(filename, L, delimiter=',', fmt='%.6e')
    print(f"Inductance matrix saved to {filename}")

def main():
    # Input and output file names
    capacitance_matrix_file = "capacitance_matrix.txt"
    inductance_matrix_file = "inductance_matrix.txt"
    
    # Calculate the inductance matrix
    L = calculate_inductance_matrix(capacitance_matrix_file)
    
    # Print the inductance matrix
    print("Inductance Matrix (H/m):")
    print(L)
    
    # Save the inductance matrix
    save_inductance_matrix(L, inductance_matrix_file)
    
    # Calculate and print the speed of propagation
    v = 1 / np.sqrt(np.mean(np.diag(L)) * np.mean(np.diag(np.linalg.inv(L))))
    print(f"\nCalculated speed of propagation: {v:.3e} m/s")
    print(f"Speed of light in vacuum: {constants.c:.3e} m/s")
    print(f"Ratio (v/c): {v/constants.c:.6f}")

if __name__ == "__main__":
    main()

Inductance Matrix (H/m):
[[9.27215041 9.2721501  9.27214781 9.27214588 9.2721458  9.27214398
  9.27214258 9.27214207 9.27214203 9.27214361]
 [9.2721501  9.27215018 9.27214782 9.27214586 9.27214577 9.27214395
  9.27214255 9.27214204 9.272142   9.27214357]
 [9.27214781 9.27214782 9.27214584 9.27214381 9.2721437  9.27214187
  9.27214046 9.27213995 9.27213991 9.27214149]
 [9.27214588 9.27214586 9.27214381 9.27214217 9.27214199 9.27214013
  9.27213872 9.2721382  9.27213816 9.27213973]
 [9.2721458  9.27214577 9.2721437  9.27214199 9.27214219 9.27214026
  9.27213882 9.2721383  9.27213825 9.27213982]
 [9.27214398 9.27214395 9.27214187 9.27214013 9.27214026 9.27213872
  9.27213721 9.27213666 9.2721366  9.27213817]
 [9.27214258 9.27214255 9.27214046 9.27213872 9.27213882 9.27213721
  9.27213608 9.27213546 9.27213538 9.27213694]
 [9.27214207 9.27214204 9.27213995 9.2721382  9.2721383  9.27213666
  9.27213546 9.27213523 9.27213507 9.27213661]
 [9.27214203 9.272142   9.27213991 9.27213816 9.2721382

In [20]:
import numpy as np
from scipy import constants

def calculate_inductance_matrix(C):
    epsilon_0 = constants.epsilon_0
    mu_0 = constants.mu_0
    return mu_0 * epsilon_0 * np.linalg.inv(C)

# Load the capacitance matrix
C = np.array([
    [3.012391e-11, -2.360589e-11, -3.033546e-12, -1.468381e-12, -8.241938e-13, -4.842981e-13, -2.902305e-13, -1.776564e-13, -1.150479e-13, -1.246688e-13],
    [-2.360589e-11, 4.882892e-11, -2.131418e-11, -1.931864e-12, -8.541966e-13, -4.652125e-13, -2.715640e-13, -1.646887e-13, -1.062942e-13, -1.150366e-13],
    [-3.033546e-12, -2.131418e-11, 4.916220e-11, -2.113876e-11, -1.839913e-12, -8.006708e-13, -4.352052e-13, -2.574928e-13, -1.647380e-13, -1.776938e-13],
    [-1.468381e-12, -1.931864e-12, -2.113876e-11, 4.922067e-11, -2.108088e-11, -1.814892e-12, -7.892876e-13, -4.349710e-13, -2.714944e-13, -2.901352e-13],
    [-8.241938e-13, -8.541966e-13, -1.839913e-12, -2.108088e-11, 4.924126e-11, -2.107971e-11, -1.813396e-12, -8.000002e-13, -4.649593e-13, -4.840151e-13],
    [-4.842981e-13, -4.652125e-13, -8.006708e-13, -1.814892e-12, -2.107971e-11, 4.924973e-11, -2.108604e-11, -1.839695e-12, -8.546352e-13, -8.245747e-13],
    [-2.902305e-13, -2.715640e-13, -4.352052e-13, -7.892876e-13, -1.813396e-12, -2.108604e-11, 4.921519e-11, -2.112611e-11, -1.933806e-12, -1.469547e-12],
    [-1.776564e-13, -1.646887e-13, -2.574928e-13, -4.349710e-13, -8.000002e-13, -1.839695e-12, -2.112611e-11, 4.913197e-11, -2.129848e-11, -3.032873e-12],
    [-1.150479e-13, -1.062942e-13, -1.647380e-13, -2.714944e-13, -4.649593e-13, -8.546352e-13, -1.933806e-12, -2.129848e-11, 4.881366e-11, -2.360420e-11],
    [-1.246688e-13, -1.150366e-13, -1.776938e-13, -2.901352e-13, -4.840151e-13, -8.245747e-13, -1.469547e-12, -3.032873e-12, -2.360420e-11, 3.012274e-11]
])

# Calculate the inductance matrix
L_calculated = calculate_inductance_matrix(C)

# Print the calculated inductance matrix
print("Calculated Inductance Matrix (H/m):")
print(L_calculated)

# Compare with the provided inductance matrix
L_provided = np.array([
    [9.272150e+00, 9.272150e+00, 9.272148e+00, 9.272146e+00, 9.272146e+00, 9.272144e+00, 9.272143e+00, 9.272142e+00, 9.272142e+00, 9.272144e+00],
    [9.272150e+00, 9.272150e+00, 9.272148e+00, 9.272146e+00, 9.272146e+00, 9.272144e+00, 9.272143e+00, 9.272142e+00, 9.272142e+00, 9.272144e+00],
    [9.272148e+00, 9.272148e+00, 9.272146e+00, 9.272144e+00, 9.272144e+00, 9.272142e+00, 9.272140e+00, 9.272140e+00, 9.272140e+00, 9.272141e+00],
    [9.272146e+00, 9.272146e+00, 9.272144e+00, 9.272142e+00, 9.272142e+00, 9.272140e+00, 9.272139e+00, 9.272138e+00, 9.272138e+00, 9.272140e+00],
    [9.272146e+00, 9.272146e+00, 9.272144e+00, 9.272142e+00, 9.272142e+00, 9.272140e+00, 9.272139e+00, 9.272138e+00, 9.272138e+00, 9.272140e+00],
    [9.272144e+00, 9.272144e+00, 9.272142e+00, 9.272140e+00, 9.272140e+00, 9.272139e+00, 9.272137e+00, 9.272137e+00, 9.272137e+00, 9.272138e+00],
    [9.272143e+00, 9.272143e+00, 9.272140e+00, 9.272139e+00, 9.272139e+00, 9.272137e+00, 9.272136e+00, 9.272135e+00, 9.272135e+00, 9.272137e+00],
    [9.272142e+00, 9.272142e+00, 9.272140e+00, 9.272138e+00, 9.272138e+00, 9.272137e+00, 9.272135e+00, 9.272135e+00, 9.272135e+00, 9.272137e+00],
    [9.272142e+00, 9.272142e+00, 9.272140e+00, 9.272138e+00, 9.272138e+00, 9.272137e+00, 9.272135e+00, 9.272135e+00, 9.272135e+00, 9.272137e+00],
    [9.272144e+00, 9.272144e+00, 9.272141e+00, 9.272140e+00, 9.272140e+00, 9.272138e+00, 9.272137e+00, 9.272137e+00, 9.272137e+00, 9.272139e+00]
])

# Calculate the maximum absolute difference
max_diff = np.max(np.abs(L_calculated - L_provided))
print(f"\nMaximum absolute difference between calculated and provided inductance matrices: {max_diff:.6e}")

# Check if the difference is within an acceptable tolerance
tolerance = 1e-6
if max_diff < tolerance:
    print("The provided inductance matrix is correct within the specified tolerance.")
else:
    print("The provided inductance matrix differs significantly from the calculated one.")

Calculated Inductance Matrix (H/m):
[[9.27215041 9.2721501  9.27214781 9.27214588 9.2721458  9.27214398
  9.27214258 9.27214207 9.27214203 9.27214361]
 [9.2721501  9.27215018 9.27214782 9.27214586 9.27214577 9.27214395
  9.27214255 9.27214204 9.272142   9.27214357]
 [9.27214781 9.27214782 9.27214584 9.27214381 9.2721437  9.27214187
  9.27214046 9.27213995 9.27213991 9.27214149]
 [9.27214588 9.27214586 9.27214381 9.27214217 9.27214199 9.27214013
  9.27213872 9.2721382  9.27213816 9.27213973]
 [9.2721458  9.27214577 9.2721437  9.27214199 9.27214219 9.27214026
  9.27213882 9.2721383  9.27213825 9.27213982]
 [9.27214398 9.27214395 9.27214187 9.27214013 9.27214026 9.27213872
  9.27213721 9.27213666 9.2721366  9.27213817]
 [9.27214258 9.27214255 9.27214046 9.27213872 9.27213882 9.27213721
  9.27213608 9.27213546 9.27213538 9.27213694]
 [9.27214207 9.27214204 9.27213995 9.2721382  9.2721383  9.27213666
  9.27213546 9.27213523 9.27213507 9.27213661]
 [9.27214203 9.272142   9.27213991 9.2721381

In [21]:
import numpy as np
from math import ceil

# Constants
f_max = 1e9  # 1 GHz
length = 0.015  # 20 cm
c = 3e8  # Speed of light
mu_0 = 4 * np.pi * 1e-7  # Permeability of free space
sigma = 5.96e7  # Conductivity of copper
tan_delta = 0.001  # Assumed loss tangent
w = 0.001  # Assumed conductor width (1 mm)

# Load matrices
C_matrix = np.loadtxt('capacitance_matrix.txt', delimiter=',')
L_matrix = np.loadtxt('inductance_matrix.txt', delimiter=',')

# Calculate number of PI sections
N = ceil(f_max * length / (c * 0.05))

# Calculate R and G matrices
R_matrix = np.sqrt(f_max * mu_0 / sigma) / w * np.eye(10)
G_matrix = 2 * np.pi * f_max * C_matrix * tan_delta

# Calculate per-section parameters
R_section = R_matrix * (length / N)
L_section = L_matrix * (length / N)
C_section = C_matrix * (length / N)
G_section = G_matrix * (length / N)

# Calculate coupling factors
k_matrix = np.zeros((10, 10))
for i in range(10):
    for j in range(10):
        if i != j:
            k_matrix[i][j] = L_matrix[i][j] / np.sqrt(L_matrix[i][i] * L_matrix[j][j])

# Generate SPICE file
with open('transmission_line_subcircuit1.cir', 'w') as f:
    f.write('* 10-wire PI Transmission Line Subcircuit Model\n\n')
    
    # Write subcircuit definition
    f.write('.SUBCKT TRANSMISSION_LINE ')
    for i in range(10):
        f.write(f'in{i+1} out{i+1} ')
    f.write('\n')
    
    # Write PI sections
    for section in range(N):
        # R and L components
        for i in range(10):
            f.write(f'R{section}_{i+1} node{section}_{i} node{section}_{i}_mid {R_section[i][i]:.6e}\n')
            f.write(f'L{section}_{i+1} node{section}_{i}_mid node{section+1}_{i} {L_section[i][i]:.6e}\n')
        
        # Mutual inductances
        count = 1
        for i in range(10):
            for j in range(i+1, 10):
                f.write(f'K{section}_{count} L{section}_{i+1} L{section}_{j+1} {k_matrix[i][j]:.6e}\n')
                count += 1
        
        # C and G components
        for i in range(10):
            f.write(f'C{section}_{i+1} node{section+1}_{i} 0 {C_section[i][i]:.6e}\n')
            f.write(f'G{section}_{i+1} node{section+1}_{i} 0 node{section+1}_{i} 0 {G_section[i][i]:.6e}\n')
        
        # Mutual capacitances
        count = 1
        for i in range(10):
            for j in range(i+1, 10):
                f.write(f'CM{section}_{count} node{section+1}_{i} node{section+1}_{j} {C_section[i][j]:.6e}\n')
                count += 1
    
    # Connect input and output nodes
    for i in range(10):
        f.write(f'R_in{i+1} in{i+1} node0_{i} 1e-6\n')
        f.write(f'R_out{i+1} node{N}_{i} out{i+1} 1e-6\n')
    
    f.write('.ENDS TRANSMISSION_LINE\n')

print(f"SPICE subcircuit file 'transmission_line_subcircuit.cir' has been generated with {N} PI sections.")

SPICE subcircuit file 'transmission_line_subcircuit.cir' has been generated with 1 PI sections.
